In [1]:
import numpy as np
import pandas as pd
from glob import glob

import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import binom

import mle
import flmp
from libfunc import to_table_body

import warnings
warnings.filterwarnings('ignore')

# Load Data

In [2]:
# get paths to data files
file_paths = glob("./data/*.txt")

# load all data into a single array
data = np.array([np.loadtxt(fname) for fname in file_paths])
N, M, K = data.shape

# define number of samples for each subject
n_samples = 24 

# number of samples for simulated data
N_SAMPLES = 100

# Model Distributions on Data with FLMP and MLE
Fit MLE and FLMP to the data, create the corresponding distribution and sample from these to create simulated data.

In [7]:
# compute parameters and generate samples
samples = {
    "FLMP": np.zeros((N, M, K)),
    "MLE": np.zeros((N, M, K))
}
params = []
for i, d in enumerate(data):
    # get parameters for MLE and FLMP fits
    _, c_A, c_V, std_A, std_V = mle.fit(np.ones(4), d, n_samples)
    _, tA, tV = flmp.fit(np.ones(K*2), d, n_samples, K)

    # save parameters
    params.append(np.concatenate([tA, tV, np.array([c_A, std_A, c_V, std_V])]))

    # sample from FLMP
    p_A, p_V, p_AV = flmp.compute_probs(tA, tV)
    samples["FLMP"][i] = binom.rvs(N_SAMPLES, np.vstack([p_A.T, p_V.T, p_AV]))
    
    # sample from MLE
    p_A, p_V, p_AV = mle.compute_probs(c_A, c_V, np.exp(std_A), np.exp(std_V))
    samples["MLE"][i]  = binom.rvs(N_SAMPLES, np.vstack([p_A.T, p_V.T, p_AV]))

Parameters
Subject 1 & -3.31 & -2.29 & -1.52 & 2.46 & 3.21 & -2.97 & 0.84 & 2.91 & 3.58 & 3.83 & 3.31 & -0.10 & 1.86 & -0.13\\
Subject 2 & -3.68 & -1.83 & -0.32 & 2.58 & 4.83 & -5.29 & 0.91 & 3.20 & 5.09 & 5.87 & 3.02 & -0.27 & 1.98 & -0.53\\
Subject 3 & -2.65 & -1.74 & -0.69 & 2.62 & 5.41 & -2.00 & 0.25 & 1.54 & 2.37 & 2.62 & 2.90 & -0.10 & 2.05 & 0.13\\
Subject 4 & -4.12 & -3.11 & -2.04 & 2.25 & 4.62 & -2.52 & -1.19 & -0.31 & 1.26 & 2.12 & 3.36 & -0.36 & 3.11 & 0.13\\
Subject 5 & -5.43 & -3.93 & -1.42 & 2.92 & 6.24 & -3.01 & 0.42 & 1.73 & 2.42 & 3.05 & 3.32 & -0.53 & 2.07 & -0.08

Simulated data for subject 1 (ignore first column)
Subject 1 & 3 & 11 & 21 & 93 & 96 & 0 & 5 & 40 & 84 & 94\\
Subject 2 & 3 & 71 & 92 & 95 & 98 & 18 & 64 & 92 & 98 & 100\\
Subject 3 & 0 & 0 & 1 & 36 & 57 & 1 & 4 & 24 & 44 & 76\\
Subject 4 & 12 & 19 & 27 & 98 & 98 & 7 & 17 & 47 & 69 & 96\\
Subject 5 & 38 & 59 & 85 & 99 & 100 & 19 & 47 & 73 & 95 & 99\\
Subject 6 & 53 & 79 & 87 & 100 & 100 & 51 & 88 & 97 & 98 

# Fit FLMP and MLE on Simulated Data

In [8]:
tmp = {
    "Fitting Model": [],
    "Sampling Model": [], 
    "Subject": [],
    "NLL": []
}
for model in samples:
    for i, Subject in enumerate(samples[model]):
        # compute negative log-likelihood for MLE fit on both simulated data
        nll, _, _, _, _ = mle.fit(np.ones(4), Subject, N_SAMPLES)
        tmp["Fitting Model"].append("MLE")
        tmp["Sampling Model"].append(model)
        tmp["Subject"].append(f"{i+1}")
        tmp["NLL"].append(nll)
        
        # compute negative log-likelihood for FLMP fit on both simulated data
        nll, _, _ = flmp.fit(np.ones(K*2), Subject, N_SAMPLES, K)
        tmp["Fitting Model"].append("FLMP")
        tmp["Sampling Model"].append(model)
        tmp["Subject"].append(f"{i+1}")
        tmp["NLL"].append(nll)

In [17]:
df_fit = pd.DataFrame.from_dict(tmp).set_index(["Fitting Model", "Subject"])
df_fit

,,Sampling Model,NLL
Fitting Model,Subject,,
MLE,1,FLMP,201.429598
FLMP,1,FLMP,65.431552
MLE,2,FLMP,146.305906
FLMP,2,FLMP,47.988344
MLE,3,FLMP,161.335452
FLMP,3,FLMP,76.731205
MLE,4,FLMP,178.334900
FLMP,4,FLMP,60.914083
MLE,5,FLMP,117.104041


# Print out stuff for Latex report

In [49]:
# print out parameters in latex format
print('Parameters for all subjects')
print(to_table_body(params))

# display data for a single subject
SUBJECT_NUMBER = 0
print(f"\nSimulated data for 'Subject {SUBJECT_NUMBER+1}'\n(ignore first column)")
print(to_table_body(np.hstack([
    samples['FLMP'][SUBJECT_NUMBER], 
    samples['MLE'][SUBJECT_NUMBER]
]), num_float=False).replace('Subject ', ''))

# extract dataframe for the single subject of negative log-likelihoods
print(f'\nNegative log-likelihoods for "Subject {SUBJECT_NUMBER+1}"')
df_fit_ = df_fit.reset_index()
df_fit_ = df_fit_[df_fit_['Subject'].astype(int) == SUBJECT_NUMBER + 1].set_index(["Fitting Model"])
df_fit_[df_fit_.columns[df_fit_.columns.isin(['Sampling Model', 'NLL'])]]

Parameters for all subjects
Subject 1 & -3.31 & -2.29 & -1.52 & 2.46 & 3.21 & -2.97 & 0.84 & 2.91 & 3.58 & 3.83 & 3.31 & -0.10 & 1.86 & -0.13\\
Subject 2 & -3.68 & -1.83 & -0.32 & 2.58 & 4.83 & -5.29 & 0.91 & 3.20 & 5.09 & 5.87 & 3.02 & -0.27 & 1.98 & -0.53\\
Subject 3 & -2.65 & -1.74 & -0.69 & 2.62 & 5.41 & -2.00 & 0.25 & 1.54 & 2.37 & 2.62 & 2.90 & -0.10 & 2.05 & 0.13\\
Subject 4 & -4.12 & -3.11 & -2.04 & 2.25 & 4.62 & -2.52 & -1.19 & -0.31 & 1.26 & 2.12 & 3.36 & -0.36 & 3.11 & 0.13\\
Subject 5 & -5.43 & -3.93 & -1.42 & 2.92 & 6.24 & -3.01 & 0.42 & 1.73 & 2.42 & 3.05 & 3.32 & -0.53 & 2.07 & -0.08

Simulated data for 'Subject 1'
(ignore first column)
1 & 3 & 11 & 21 & 93 & 96 & 0 & 5 & 40 & 84 & 94\\
2 & 3 & 71 & 92 & 95 & 98 & 18 & 64 & 92 & 98 & 100\\
3 & 0 & 0 & 1 & 36 & 57 & 1 & 4 & 24 & 44 & 76\\
4 & 12 & 19 & 27 & 98 & 98 & 7 & 17 & 47 & 69 & 96\\
5 & 38 & 59 & 85 & 99 & 100 & 19 & 47 & 73 & 95 & 99\\
6 & 53 & 79 & 87 & 100 & 100 & 51 & 88 & 97 & 98 & 100\\
7 & 63 & 87 & 95 & 99

,Sampling Model,NLL
Fitting Model,,
MLE,FLMP,201.429598
FLMP,FLMP,65.431552
MLE,MLE,74.494720
FLMP,MLE,83.264990
